# Sketchpad - agents

This notebook was used to set up the proper workflow of the final pipeline.

This script lacks proper documentation, and is largely kept for archival purposes.

In [1]:
# imports
import faiss, json, tiktoken, openai, os, re, ast
import google.generativeai as genai
import numpy as np
import geopandas as gpd
import shapely
from pathlib import Path
from dotenv import load_dotenv
from shapely.ops import unary_union
from shapely.geometry import Polygon, MultiPolygon, GeometryCollection
from typing import Any, Dict, List, Optional, Tuple

from utils import cut_polygon
from agent_toolkit import (
    AgentOrchestrator,
    ReasoningAgent,
    ValidationAgent,
    call_llm,
    embed_texts,
    faiss_search,
    faiss_search_flatten,
    gpkg_query,
    parse_llm_json,
    prepare_system_prompt,
    sysprompt_validator_template,
)

load_dotenv()

/home/fkok2/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### loading RAG and GEO databases

In [2]:
# load FAISS database
EMB_DIM = 3072
EMBEDDER = "text-embedding-3-large"

FAISS_INDEX_DIR = "data/spatial_genai_storage/database_RAG"
FAISS_META_PATH = Path(FAISS_INDEX_DIR) / "metadata.json"
FAISS_INDEX_PATH = str(Path(FAISS_INDEX_DIR) / "faiss.index")

In [5]:
# load geopackage data

GPKG_FILE_PATH = 'data/spatial_genai_storage/data_PDOK/top10nl_Compleet.gpkg'
BBOX_UTRECHT_PROV = (109311, 430032, 169326, 479261)

In [ ]:
# grab the province of Utrecht polygon as base polygon
base_polygon = gpkg_query(
    table_name='top10nl_registratief_gebied_vlak',
    bbox=BBOX_UTRECHT_PROV,
    filters={'typeregistratiefgebied': 'provincie', 'naamnl': 'Utrecht'}  # Adjust column if name is in 'naamofficieel'
)
base_geom = base_polygon.geometry.iloc[0]  # Extract Shapely geometry

### Setting up API calling

In [8]:
api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=api_key)

### Setting up prompts

In [ ]:
# TO-DO: swap context to bottom with instruction at top, see if improved functioning

sysprompt_inhoudsexpert_reasoning = """
Jij bent een **inhoudsexpert op het gebied van ruimtelijke ordening** en juridische analyse, gespecialiseerd in het identificeren van ruimtelijke belemmeringen voor nieuwe plannen binnen de provincie Utrecht. Jouw taak is het samenstellen van een **kritische, goed beargumenteerde lijst van belemmeringen** ("lijst van belemmeringen") op basis van juridische informatie en een database-overzicht met polygonen.

**Je werkt in twee stappen:**  
Deze prompt is bedoeld voor de **eerste stap**. Je geeft per gevonden belemmering een beknopte, maar duidelijke redenatie voor je keuze. Het uiteindelijke doel is te bepalen welke gebieden van de basispolygoon ("{base_polygon}") niet geschikt zijn voor plaatsing van het opgegeven object ("{thematic_object}").

### **Jouw context**  
Je ontvangt de volgende input:
- **thematic_object**: {thematic_object}
- **object_description**: {object_description}
- **judicial_reference**: {judicial_reference}
- **database_reference**: {database_reference}
- **base_polygon**: {base_polygon}

### **Jouw opdracht**

1. **Analyseer** zorgvuldig de juridische referenties en de database_reference. 
2. **Redeneer** kritisch met gebruik van je eigen kennis van ruimtelijke ordening en juridische kaders ook wanneer deze niet expliciet in de juridische referenties staan.
3. **Identificeer** voor het opgegeven thematische object ("thematic_object") en de bijbehorende beschrijving ("object_description") alle relevante belemmerende gebieden in de database (dus: specifieke combinaties van tabel, kolom en waarde).
4. **Categoriseer** elke gevonden kolom:waarde-combinatie met een van de volgende categorieën:
    - **Harde belemmering**: het is op basis van de juridische informatie en objectbeschrijving met zekerheid niet toegestaan om het thematische object daar te plaatsen.
    - **Complexe belemmering**: er is mogelijk een belemmering, maar dit hangt af van nadere details van het thematische object.
    - **Zachte belemmering**: er is mogelijk een kleine belemmering, maar doorgaans is plaatsing waarschijnlijk wel toegestaan.
5. **Wees kritisch**: niet elke tabel of rij hoeft een belemmering te zijn. Maak onderscheid tussen duidelijke, mogelijke en marginale belemmeringen.
6. **Geef per gevonden belemmering een korte, duidelijke redenatie** waarom deze categorie is gekozen.

### **Outputformaat**

Voor elke gevonden belemmering geef je de volgende gestructureerde output:

```json
[
  {
    "categorie": "<harde|complexe|zachte belemmering>",
    "tabel": "<tabelnaam>",
    "kolom": "<kolomnaam>",
    "waarde": "<waarde>",
    "redenatie": "<korte, kritische redenatie>"
  },
  ...
]
```

- Gebruik altijd het bovenstaande JSON-formaat.
- Voeg alleen belemmeringen toe waar je op basis van de input daadwerkelijk een argument voor hebt.
- Houd je redenaties kort en to-the-point.

### **Let op**

- Voeg **geen** belemmeringen toe zonder duidelijke juridische of feitelijke onderbouwing uit de context.
- **Voorkom overgeneralisatie**: niet alles is een belemmering.
- Je antwoord wordt gebruikt als input voor een vervolgagent die ruimtelijke bewerkingen uitvoert; precisie is belangrijk.

---

**Begin nu met het analyseren van de input en formuleer per gevonden belemmering de redenatie en categorie in bovenstaand formaat.**
"""

### Basic workflow

In [13]:
# Get column:feature filters plus reasoning

NUM_ROUNDS = 2

# basic inputs
thematic_object = "windturbine"
object_description = "Er is uitgegaan van een ‘referentie windturbinetype’. Uitgangspunt is dat het referentie windturbinetype een ashoogte van 160 meter, rotordiameter van 162 meter en tiphoogte van 241 meter heeft."
base_polygon_name = "Provincie Utrecht"

# get judicial references from RAG database
query = f"Vind juridische belemmeringen voor het plaatsen van een {thematic_object} met de volgende kenmerken: {object_description}. Geef relevante wet- en regelgeving, beleidsdocumenten en ruimtelijke plannen binnen {base_polygon_name}."
judicial_reference = faiss_search(query=query, k=30, location="Utrecht")
judicial_reference = faiss_search_flatten(judicial_reference)

# get database reference from pre-made file
database_reference_path = "data/spatial_genai_storage/data_PDOK/llm_reference.txt"
with open(database_reference_path, "r", encoding="utf-8") as f:
    database_reference = f.read()

# prepare agent ecosystem
reasoning_agent = ReasoningAgent(
    name="inhoudsexpert",
    system_prompt_template=sysprompt_inhoudsexpert_reasoning,
)
validation_agent = ValidationAgent(
    name="kwaliteitscontroleur",
    system_prompt_template=sysprompt_validator_template,
)
orchestrator = AgentOrchestrator(reasoning_agent, validation_agent)

agent_context = {
    "thematic_object": thematic_object,
    "object_description": object_description,
    "judicial_reference": judicial_reference,
    "database_reference": database_reference,
    "base_polygon_name": base_polygon_name,
    "feedback": [],
}

agent_rounds = orchestrator.run_rounds(agent_context, num_rounds=NUM_ROUNDS)
latest_round = agent_rounds[-1]
filters_plus_reasoning = latest_round["reasoning"]["raw"]
filters = latest_round["reasoning"]["parsed"]
validation_summary = latest_round["validation"]["parsed"]

Searching for: 'Vind juridische belemmeringen voor het plaatsen van een windturbine met de volgende kenmerken: Er is uitgegaan van een ‘referentie windturbinetype’. Uitgangspunt is dat het referentie windturbinetype een ashoogte van 160 meter, rotordiameter van 162 meter en tiphoogte van 241 meter heeft.. Geef relevante wet- en regelgeving, beleidsdocumenten en ruimtelijke plannen binnen Provincie Utrecht.' (location filter: Utrecht)


In [14]:
# Inspect validation feedback from the latest round
validation_summary

{'approved': False,
 'comments': ['Het voorstel is een verbetering en bevat een relevante set aan belemmeringen met een logische categorisering.',
  'De redeneringen zijn inhoudelijk sterk en juridisch/ruimtelijk relevant.',
  'Het kernprobleem uit de vorige ronde, het systematisch ontbreken van bufferzones, is echter niet opgelost in de filterdefinities, wat de analyse onbetrouwbaar maakt.'],
 'issues': ['Bufferzones ontbreken systematisch: De redenering voor objecten als gebouwen, woonkernen, wegen, spoorlijnen en erfgoed noemt terecht bufferzones (i.v.m. geluid, slagschaduw, externe veiligheid, visuele impact), maar de filters zelf implementeren deze niet. Ze filteren alleen de objecten zelf. Suggestie: specificeer voor elke complexe belemmering de vereiste buffer-afstand en pas deze toe in de analyse.',
  "Luchtvaartbeperkingen onvolledig: Het filter 'vliegverkeer' dekt alleen start- en landingsbanen. Voor een turbine van 241m zijn radarverstoringsgebieden (met name van de defensie

In [15]:
print(filters_plus_reasoning)

```json
[
  {
    "categorie": "harde belemmering",
    "tabel": "top10nl_functioneel_gebied_multivlak",
    "kolom": "soortnaam",
    "waarde": "natura2000-gebied",
    "redenatie": "Natura 2000-gebieden hebben de hoogste wettelijke beschermingsstatus. Windturbines vormen een significant risico voor beschermde vogel- en vleermuissoorten, waardoor plaatsing hier juridisch en ecologisch vrijwel uitgesloten is."
  },
  {
    "categorie": "harde belemmering",
    "tabel": "top10nl_functioneel_gebied_multivlak",
    "kolom": "typefunctioneelgebied",
    "waarde": "nationaal park",
    "redenatie": "Nationale parken zijn kerngebieden van het Natuurnetwerk Nederland (NNN) met een sterke focus op natuur en landschap. Een windturbine van 241m hoog is onverenigbaar met de kernkwaliteiten en beschermingsdoelen."
  },
  {
    "categorie": "harde belemmering",
    "tabel": "top10nl_functioneel_gebied_multivlak",
    "kolom": "typefunctioneelgebied",
    "waarde": "natuurgebied",
    "redenatie": "

### Start processing (takes a while, 15-20min... with bigger boundary box 35-45min)

In [17]:
# Cut from province polygon using json format filters

categories = {
    "harde belemmering": [],
    "complexe belemmering": [],
    "zachte belemmering": []
}

for item in filters:
    categorie = item.get("categorie", "").lower()
    if categorie in categories:
        categories[categorie].append(item)
        
# Store cut geometries per category
cut_areas_by_category = {cat: [] for cat in categories.keys()}

# Process all filters and apply to base geometry
for item in filters:
    tabel = item["tabel"]
    kolom = item["kolom"]
    waarde = item["waarde"]
    categorie = item.get("categorie", "").lower()

    print(f"Processing filter: {tabel}.{kolom} = '{waarde}'")
    cut_gdf = gpkg_query(table_name=tabel, filters={kolom: waarde})
    
    if not cut_gdf.empty:
        cut_geoms = cut_gdf.geometry.tolist()
        union_cut = unary_union(cut_geoms)
        
        if categorie in cut_areas_by_category:
            cut_areas_by_category[categorie].append(union_cut)
        
        base_geom = cut_polygon(base_geom, union_cut)
        
        print(f"Cut applied for {tabel}.{kolom}='{waarde}' - {len(cut_gdf)} polygons removed.")
    else:
        print(f"No polygons found for {tabel}.{kolom}='{waarde}' - skipping.")

# Create new GeoDataFrame with cut geometry
cut_gdf_result = gpd.GeoDataFrame(
    {"name": ["Utrecht_Province_Cut"], "description": ["Remaining area after all cuts"]},
    geometry=[base_geom],
    crs=base_polygon.crs
)

# Optional: Save to file
output_file = "utrecht_cut_with_categories.gpkg"
cut_gdf_result.to_file(output_file, layer="remaining_area", driver="GPKG")
print(f"Remaining area saved to {output_file}")

Processing filter: top10nl_functioneel_gebied_multivlak.soortnaam = 'natura2000-gebied'
Cut applied for top10nl_functioneel_gebied_multivlak.soortnaam='natura2000-gebied' - 8 polygons removed.
Processing filter: top10nl_functioneel_gebied_multivlak.typefunctioneelgebied = 'nationaal park'
Cut applied for top10nl_functioneel_gebied_multivlak.typefunctioneelgebied='nationaal park' - 1 polygons removed.
Processing filter: top10nl_functioneel_gebied_multivlak.typefunctioneelgebied = 'natuurgebied'
Cut applied for top10nl_functioneel_gebied_multivlak.typefunctioneelgebied='natuurgebied' - 16 polygons removed.
Processing filter: top10nl_wegdeel_vlak.hoofdverkeersgebruik = 'vliegverkeer'
Cut applied for top10nl_functioneel_gebied_multivlak.typefunctioneelgebied='natuurgebied' - 16 polygons removed.
Processing filter: top10nl_wegdeel_vlak.hoofdverkeersgebruik = 'vliegverkeer'
Cut applied for top10nl_wegdeel_vlak.hoofdverkeersgebruik='vliegverkeer' - 85 polygons removed.
Processing filter: top1

In [18]:
output_file = "utrecht_cut_with_categories.gpkg"
cut_gdf_result.to_file(output_file, layer="remaining_area", driver="GPKG")
print(f"Remaining area saved to {output_file}")

Remaining area saved to utrecht_cut_with_categories.gpkg


In [19]:
# Save cut areas by category as separate layers
for categorie, cut_geoms in cut_areas_by_category.items():
    if not cut_geoms:
        print(f"No cut areas for category: {categorie}")
        continue
    
    # Union all cut geometries for this category
    category_union = unary_union(cut_geoms)
    
    if category_union.is_empty:
        print(f"Empty geometry for category: {categorie}")
        continue
    
    # Create layer name
    layer_name = f"cut_{categorie.replace(' ', '_')}"
    
    # Create GeoDataFrame for cut areas
    cut_area_gdf = gpd.GeoDataFrame(
        {
            "name": [f"Cut areas - {categorie}"],
            "description": [f"Areas removed due to {categorie}"],
            "categorie": [categorie]
        },
        geometry=[category_union],
        crs=base_polygon.crs
    )
      
    # Save to GeoPackage
    cut_area_gdf.to_file(output_file, layer=layer_name, driver="GPKG", mode='a')
    print(f"Saved cut areas for '{categorie}' as layer '{layer_name}'")

print(f"\nAll layers saved to {output_file}")

Saved cut areas for 'harde belemmering' as layer 'cut_harde_belemmering'
Saved cut areas for 'complexe belemmering' as layer 'cut_complexe_belemmering'
Saved cut areas for 'zachte belemmering' as layer 'cut_zachte_belemmering'

All layers saved to utrecht_cut_with_categories.gpkg
Saved cut areas for 'complexe belemmering' as layer 'cut_complexe_belemmering'
Saved cut areas for 'zachte belemmering' as layer 'cut_zachte_belemmering'

All layers saved to utrecht_cut_with_categories.gpkg


In [ ]:
# EXPERIMENTAL, SIMPLIFY INCLUDING LAYERS

# Simplify and validate all layers (remaining area + cut areas by category)

input_file = "utrecht_cut_with_categories.gpkg"
output_file = "utrecht_cut_with_categories_simplified_2.gpkg"

# Get all layers from the input GeoPackage
layers_df = gpd.list_layers(input_file)
print(f"Found {len(layers_df)} layers")
print(f"Layer info:\n{layers_df}")

# Extract layer names from the 'name' column
layer_names = layers_df['name'].tolist()
print(f"\nLayer names: {layer_names}")

for idx, layer_name in enumerate(layer_names):
    print(f"\n=== Processing layer {idx+1}/{len(layer_names)}: {layer_name} ===")
    
    try:
        # 1) Read the layer
        gdf = gpd.read_file(input_file, layer=layer_name)
        
        if gdf.empty:
            print(f"Layer {layer_name} is empty, skipping")
            continue
        
        assert gdf.crs is not None, f"CRS missing for layer {layer_name}"
        
        # 2) Ensure polygonal, valid parts only
        gdf["geometry"] = gdf.geometry.apply(shapely.make_valid)
        gdf = gdf[~gdf.geometry.is_empty & gdf.geometry.notna()]
        
        if gdf.empty:
            print(f"No valid geometries in layer {layer_name}, skipping")
            continue
        
        # 3) Explode geometry collections
        gdf = gdf.explode(ignore_index=True)
        
        # 4) Keep only polygonal types
        gdf = gdf[gdf.geom_type.isin(["Polygon", "MultiPolygon"])]
        
        if gdf.empty:
            print(f"No polygonal geometries remain for layer {layer_name}")
            continue
        
        # 5) Union into a single polygonal geometry
        poly_union = unary_union(gdf.geometry.values)
        
        def polygons_only(geom):
            if geom.is_empty:
                return geom
            if isinstance(geom, (Polygon, MultiPolygon)):
                return geom
            if isinstance(geom, GeometryCollection):
                polys = [g for g in geom.geoms if isinstance(g, (Polygon, MultiPolygon))]
                if not polys:
                    return shapely.geometry.GeometryCollection()
                return unary_union(polys)
            return shapely.geometry.GeometryCollection()
        
        poly_union = polygons_only(poly_union)
        poly_union = shapely.make_valid(poly_union)
        
        if poly_union.is_empty:
            print(f"Empty geometry after processing for layer {layer_name}, skipping")
            continue
        
        if isinstance(poly_union, Polygon):
            poly_union = MultiPolygon([poly_union])
        
        # 6) Remove tiny slivers (< 1 m²)
        tmp = gpd.GeoDataFrame(geometry=[poly_union], crs=gdf.crs).explode(index_parts=False)
        tmp = tmp[tmp.area > 1.0]  # keep > 1 m²
        
        if tmp.empty:
            print(f"All polygons too small for layer {layer_name}, skipping")
            continue
        
        poly_union = unary_union(tmp.geometry.values)
        
        if isinstance(poly_union, Polygon):
            poly_union = MultiPolygon([poly_union])
        
        # 7) Save the simplified layer
        # Get original attributes from first row
        attrs = gdf.iloc[0].drop('geometry').to_dict()
        
        out = gpd.GeoDataFrame(
            [attrs],
            geometry=[poly_union],
            crs=gdf.crs
        )
        
        print(f"  Geometry type: {out.geom_type.iloc[0]}")
        print(f"  Area: {out.geometry.iloc[0].area:.2f} sq meters")
        print(f"  Number of parts: {len(list(out.geometry.iloc[0].geoms)) if hasattr(out.geometry.iloc[0], 'geoms') else 1}")
        
        # Save (first layer creates file, others append)
        if idx == 0:
            out.to_file(output_file, layer=layer_name, driver="GPKG")
        else:
            out.to_file(output_file, layer=layer_name, driver="GPKG", mode='a')
        
        print(f"  ✓ Saved simplified layer '{layer_name}'")
        
    except Exception as e:
        print(f"  ✗ Error processing layer '{layer_name}': {e}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*60}")
print(f"All simplified layers saved to {output_file}")
print(f"{'='*60}")

# 8) Verify all layers
print("\nVerification summary:")
print("-" * 60)
try:
    layers_df_out = gpd.list_layers(output_file)
    for layer_name in layers_df_out['name'].tolist():
        chk = gpd.read_file(output_file, layer=layer_name)
        print(f"\nLayer: {layer_name}")
        print(f"  Records: {len(chk)}")
        print(f"  Geometry type: {chk.geom_type.iloc[0]}")
        print(f"  CRS: {chk.crs}")
        print(f"  Total area: {chk.geometry.iloc[0].area:.2f} sq meters")
except Exception as e:
    print(f"Error during verification: {e}")

Found 4 layers
Layer info:
                       name       geometry_type
0            remaining_area  GeometryCollection
1     cut_harde_belemmering        MultiPolygon
2  cut_complexe_belemmering        MultiPolygon
3    cut_zachte_belemmering        MultiPolygon

Layer names: ['remaining_area', 'cut_harde_belemmering', 'cut_complexe_belemmering', 'cut_zachte_belemmering']

=== Processing layer 1/4: remaining_area ===
  Geometry type: MultiPolygon
  Area: 1001739265.31 sq meters
  Number of parts: 998
  ✓ Saved simplified layer 'remaining_area'

=== Processing layer 2/4: cut_harde_belemmering ===
  Geometry type: MultiPolygon
  Area: 354817705.11 sq meters
  Number of parts: 121
  ✓ Saved simplified layer 'cut_harde_belemmering'

=== Processing layer 3/4: cut_complexe_belemmering ===
  Geometry type: MultiPolygon
  Area: 889981874.43 sq meters
  Number of parts: 89213
  ✓ Saved simplified layer 'cut_complexe_belemmering'

=== Processing layer 4/4: cut_zachte_belemmering ===
  Geomet